In [ ]:
import json
import pandas as pd
import numpy as np
from pathlib import Path 

import hvplot.pandas



In [259]:
# load the sleep data into a pandas dataframe
sleep_data_json = Path('../data/raw/dummy_data.json')

with open(sleep_data_json) as f:
    sleep_data = json.load(f)

df = pd.DataFrame(sleep_data)

In [ ]:
# filter by long_sleep to get nighttime sleep
df[df['type'] == 'long_sleep']

In [ ]:
# examine one night of sleep
df_1night = df[df['type'] == 'long_sleep'].iloc[5]
df_1night

In [ ]:
# 420 min worth of sleep phase
len(df_1night['sleep_phase_5_min']) * 5 

In [ ]:
# 418 min worth worth of movement #
len(df_1night['movement_30_sec']) / 2 

In [ ]:
# 418 min = 25080 sec, which matches up with 'time_in_bed' in the json file
df_1night['time_in_bed']

In [ ]:
df[df['type'] == 'long_sleep']['day'].unique()

In [ ]:
# Convert the bedtime_start to a pandas datetime object
bedtime_start = pd.to_datetime(df_1night['bedtime_start'])

# Extract the movement and sleep phase data
movement_30_sec = [int(s) for s in list(df_1night['movement_30_sec'])]
sleep_phase_5_min = [int(s) for s in list(df_1night['sleep_phase_5_min'])]

# Create a pandas timeseries with 30-second intervals
timeseries_30_sec = pd.date_range(start=bedtime_start, periods=len(movement_30_sec), freq='30s')

# Repeat the sleep phase data to match the new times array
sleep_phase_30_sec = np.repeat(sleep_phase_5_min, (5 * 60) // 30)[:len(timeseries_30_sec)]

# Create the resampled DataFrame
df_resampled = pd.DataFrame({'time': timeseries_30_sec, 'movement': movement_30_sec, 'sleep_phase': sleep_phase_30_sec})
df_resampled.loc[:,'day'] = df_1night['day']
df_resampled.set_index(['day', 'time'], inplace=True)
df_resampled

In [ ]:
# identify potential RBD events
df_resampled.loc[(df_resampled['movement'] >= 3) & (df_resampled['sleep_phase'] == 3), 'RBD'] = df_resampled['movement'] + 0.5
df_resampled.loc[(df_resampled['movement'] >= 3) & (df_resampled['sleep_phase'] == 3)]

In [ ]:
# Make a plot of the sleep phase and movement data, with potential RBD events highlighted

from holoviews import opts
from bokeh.models.formatters import DatetimeTickFormatter

# Create a DatetimeTickFormatter with the desired format
formatter = DatetimeTickFormatter(hours='%H:%M', minutes='%H:%M')

# Define the mapping of numerical values to text labels for sleep phases
sleep_phase_labels = {
    1: 'Deep Sleep',
    2: 'Light Sleep',
    3: 'REM Sleep',
    4: 'Awake'
}

# Plot the line and scatter with a secondary y-axis
line_plot = df_resampled.hvplot.line(x='time', y='sleep_phase', xformatter=formatter, label='Sleep Phase').opts(
    ylabel='Sleep Phase',
    yticks=[(k, v) for k, v in sleep_phase_labels.items()]
)
line_plot2 = df_resampled.hvplot.line(x='time', y='movement', xformatter=formatter, label='Movement').opts(
    ylabel='Movement', color='orange'
)
scatter_plot = df_resampled.hvplot.scatter(x='time', y='RBD', color='green', size=20, label='RBD Event?').opts(
    yaxis='right',
    ylabel='RBD'
)

# Combine the plots
combined_plot = line_plot * line_plot2 * scatter_plot
combined_plot.opts(
    opts.Overlay(
        yaxis='left',
        yformatter='%d'  # Adjust y-axis formatter if needed
    )
)